In [13]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
def custom_warning_format(message, category, filename, lineno, file=None, line=None):
    return f'{filename}:{lineno}: {category.__name__}: {message}\n'
from sklearn.impute import KNNImputer

# Pandas fillna con ffill o bfill
* Recomendación:

**Adecuado para datos de series temporales o donde hay una lógica clara de continuidad.
Específicamente útil cuando tienes datos ordenados cronológicamente y tiene sentido que los valores faltantes sean reemplazados por el valor más cercano en el tiempo (anterior o siguiente).**

In [14]:
import pandas as pd
import numpy as np

# Crear un DataFrame con datos de series temporales (precios de acciones)
data = {
    'date': pd.date_range(start='2023-01-01', periods=5, freq='D'),
    'stock_price': [100, np.nan, 102, np.nan, 104]
}
df = pd.DataFrame(data)

# Usar ffill (forward fill) para rellenar valores nulos con el valor anterior
df['stock_price_ffill'] = df['stock_price'].fillna(method='ffill')

# Usar bfill (backward fill) para rellenar valores nulos con el valor siguiente
df['stock_price_bfill'] = df['stock_price'].fillna(method='bfill')

print(df)


        date  stock_price  stock_price_ffill  stock_price_bfill
0 2023-01-01        100.0              100.0              100.0
1 2023-01-02          NaN              100.0              102.0
2 2023-01-03        102.0              102.0              102.0
3 2023-01-04          NaN              102.0              104.0
4 2023-01-05        104.0              104.0              104.0


C:\Users\FRANK\AppData\Local\Temp\ipykernel_20276\1307855412.py:12: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['stock_price_ffill'] = df['stock_price'].fillna(method='ffill')
C:\Users\FRANK\AppData\Local\Temp\ipykernel_20276\1307855412.py:15: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['stock_price_bfill'] = df['stock_price'].fillna(method='bfill')


# scikit-learn KNNImputer
* Recomendación:
- Adecuado para datos multidimensionales donde se desea considerar la similitud entre muestras.
Especialmente útil cuando tienes un conjunto de datos con múltiples características (atributos) y deseas usar la similitud entre estas características para imputar los valores faltantes.

- Se debe usar con las demás columnas para que tenga coherencia

In [ ]:
from sklearn.impute import KNNImputer

# Crear una instancia del KNNImputer
imputer = KNNImputer(n_neighbors=2)

# Imputar los valores nulos
df2[['historical_default',
      'customer_age',
      'customer_income',
       'term_years',
       'cred_hist_length',
       'Current_loan_status']] = imputer.fit_transform(df2[['historical_default',
                                                            'customer_age',
                                                            'customer_income',
                                                            'term_years',
                                                            'cred_hist_length',
                                                            'Current_loan_status']])

In [15]:
import pandas as pd
from sklearn.impute import KNNImputer

# Crear un DataFrame con datos multidimensionales (características de clientes)

data = {
    'customer_id': [1, 2, 3, 4, 5],
    'customer_age': [22, 21, 25, 23, 24],
    'customer_income': [59000, 9600, 9600, 65500, 54400],
    'home_ownership': ['RENT', 'OWN', 'MORTGAGE', 'RENT', 'RENT'],
    'employment_duration': [123, 5, 1, 4, 8],
    'loan_intent': ['PERSONAL', 'EDUCATION', 'MEDICAL', 'MEDICAL', 'MEDICAL'],
    'loan_grade': ['C', 'A', 'B', 'B', 'B'],
    'loan_amnt': ['£35,000.00', '£1,000.00', '£5,500.00', '£35,000.00', '£35,000.00'],
    'loan_int_rate': [16.02, 11.14, 12.87, 15.23, 14.27],
    'term_years': [10, 1, 5, 10, 10],
    'historical_default': ['Y', None, 'N', 'N', 'Y'],
    'cred_hist_length': [3, 2, 3, 2, 4],
    'Current_loan_status': ['DEFAULT', 'NO DEFAULT', 'DEFAULT', 'DEFAULT', 'DEFAULT']
}

df = pd.DataFrame(data)

# Convertir 'Y' y 'N' a 1 y 0
def convert_bin(value):
    if value == 'Y':
        return 1
    elif value == 'N':
        return 0
    return value

df['historical_default'] = df['historical_default'].apply(convert_bin)

# Crear una instancia del KNNImputer
imputer = KNNImputer(n_neighbors=2)

# Imputar los valores nulos
df[['historical_default']] = imputer.fit_transform(df[['historical_default']])

# Asegurarse de que los valores imputados son enteros (0 o 1)
df['historical_default'] = df['historical_default'].round().astype(int)

print(df)

   customer_id  customer_age  customer_income home_ownership  \
0            1            22            59000           RENT   
1            2            21             9600            OWN   
2            3            25             9600       MORTGAGE   
3            4            23            65500           RENT   
4            5            24            54400           RENT   

   employment_duration loan_intent loan_grade   loan_amnt  loan_int_rate  \
0                  123    PERSONAL          C  £35,000.00          16.02   
1                    5   EDUCATION          A   £1,000.00          11.14   
2                    1     MEDICAL          B   £5,500.00          12.87   
3                    4     MEDICAL          B  £35,000.00          15.23   
4                    8     MEDICAL          B  £35,000.00          14.27   

   term_years  historical_default  cred_hist_length Current_loan_status  
0          10                   1                 3             DEFAULT  
1         

In [16]:
import pandas as pd

# df2 = df2.sort_values(by='customer_id').reset_index(drop=True) # ordenar el dataframe

data = {
    'customer_id': [1, 2, 3, None, 5],
    'customer_age': [22, 21, 25, 23, 24],
    'customer_income': [59000, 9600, 9600, 65500, 54400]
}
df2 = pd.DataFrame(data)

print("Antes de reasignar IDs:")
print(df2)


Antes de reasignar IDs:
   customer_id  customer_age  customer_income
0          1.0            22            59000
1          2.0            21             9600
2          3.0            25             9600
3          NaN            23            65500
4          5.0            24            54400


In [17]:
# Asignar nuevos IDs secuenciales a la columna customer_id
df2['customer_id'] = range(1, len(df2) + 1)

print("Después de reasignar IDs:")
print(df2)



Después de reasignar IDs:
   customer_id  customer_age  customer_income
0            1            22            59000
1            2            21             9600
2            3            25             9600
3            4            23            65500
4            5            24            54400
